In [1]:
%pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 78.3 MB/s eta 0:00:00


In [3]:
def extract_text_from_pdf(pdf_file_path):
 import fitz
 doc = fitz.open(pdf_file_path)
 text=""
 for page in doc:
  text+=page.get_text()
  return text


In [4]:
# Replace 'your_uploaded_file.pdf' with the actual path to your PDF file
pdf_file_path = 'RaviKumarJhaSDE2025.pdf'
extracted_text = extract_text_from_pdf(pdf_file_path)
print(extracted_text)

Ravi Kumar Jha
+91 9799838629 | iamravijha057@gmail.com |
https://www.linkedin.com/in/ravi-jha-a08266151/ | https://github.com/ravijha9546 |
https://leetcode.com/u/Ravijha9546/
Greater Noida, Uttar Pradesh - 201310, India
OBJECTIVE
A highly motivated and detail-oriented Computer Science graduate with a strong foundation in software
development, data structures, operating systems such as Linux. Looking for an entry-level Software Development
Engineer (SDE) position to leverage my programming skills, problem-solving abilities, and passion for building
scalable and efficient software solutions. Interested in contributing to innovative projects while continually learning
and growing within a dynamic engineering team.
SKILLS
• Programming Languages:C, C++, JavaScript
• Web Technologies:HTML, CSS, Node.js, Express.js, React
• Cloud Technologies:Amazon Web Services, Microsoft Azure
• DevOps & Version Control:Git, GitHub, Docker, CI/CD , Terraform
• Core Subject:Data Structures and Algorithms,

In [5]:
%pip install transformers torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 746.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [7]:
from transformers import pipeline
import re
import json

def parse_resume_with_huggingface(resume_text):
    """
    Parses resume text using a Hugging Face NER model to extract structured data.

    Args:
        resume_text (str): The raw text of the resume.

    Returns:
        dict: A dictionary containing the extracted information.
    """
    # Load a pre-trained NER model via the Hugging Face pipeline.
    # "dslim/bert-base-NER" is a popular and effective model for general NER tasks.
    # `grouped_entities=True` helps to combine multi-word entities (e.g., "John Doe" instead of "John" and "Doe").
    print("Loading NER model...")
    ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)
    print("Model loaded. Parsing text...")

    # Run the text through the NER pipeline.
    ner_results = ner_pipeline(resume_text)

    # --- 4. Process NER Results into a Structured Format ---
    # The output from the NER model is a list of entities. We need to process
    # this list to build a structured JSON-like object.

    structured_data = {
        "name": None,
        "organizations": [],
        "locations": [],
        "miscellaneous": [],
        "contact": {
            "email": None,
            "phone": None
        }
    }

    # A set to avoid duplicate entries for organizations and locations
    unique_orgs = set()
    unique_locs = set()
    unique_misc = set()

    person_names = []

    for entity in ner_results:
        entity_group = entity['entity_group']
        word = entity['word']

        # The model uses specific tags: PER (Person), ORG (Organization), LOC (Location), MISC (Miscellaneous)
        if entity_group == 'PER':
            person_names.append(word)
        elif entity_group == 'ORG':
            # Clean up common junk words that might be tagged as organizations
            if len(word) > 2 and "http" not in word:
                 unique_orgs.add(word)
        elif entity_group == 'LOC':
            unique_locs.add(word)
        elif entity_group == 'MISC':
            unique_misc.add(word)

    # Heuristic: The first full person name found is likely the candidate's name.
    if person_names:
        structured_data["name"] = " ".join(person_names[0].split()) if person_names else None


    structured_data["organizations"] = sorted(list(unique_orgs))
    structured_data["locations"] = sorted(list(unique_locs))
    structured_data["miscellaneous"] = sorted(list(unique_misc))


    # --- 5. Use Regular Expressions for Contact Info (NER models are not always best for this) ---
    # Regex can be more reliable for specific patterns like email and phone numbers.
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_regex = r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'

    email = re.search(email_regex, resume_text)
    phone = re.search(phone_regex, resume_text)

    if email:
        structured_data["contact"]["email"] = email.group(0)
    if phone:
        structured_data["contact"]["phone"] = phone.group(0)

    print("Parsing complete.")
    return structured_data


# --- 6. Example Usage ---
if __name__ == "__main__":
  print("Parsing sample text")
  parsed_sample_text = parse_resume_with_huggingface(extracted_text)
  print("Parisng done")
  print(json.dumps(parsed_sample_text, indent=2))

Parsing sample text
Loading NER model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:181: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Model loaded. Parsing text...
Parsing complete.
Parisng done
{
  "name": "Ravi Kumar J",
  "organizations": [
    "##H",
    "Amazon Web Services",
    "Computer Networks",
    "Computer Science",
    "Development",
    "Express",
    "Microsoft Azure"
  ],
  "locations": [
    "Greater Noida",
    "India",
    "Uttar Pradesh"
  ],
  "miscellaneous": [
    "C",
    "Java",
    "Linux"
  ],
  "contact": {
    "email": "iamravijha057@gmail.com",
    "phone": "9799838629"
  }
}


# Task
Push the current Google Colab notebook to a specified GitHub repository.

## Connect colab to github

### Subtask:
Link your Google Colab account to your GitHub account.


## Save a copy of the notebook to github

### Subtask:
Save the current notebook to a GitHub repository.


## Commit and push changes

### Subtask:
Make changes to the notebook and push the updates to the linked GitHub repository.


**Reasoning**:
Add a comment to a code cell to make a minor edit to the notebook as required by the instructions.



In [8]:
# Replace 'your_uploaded_file.pdf' with the actual path to your PDF file
pdf_file_path = 'RaviKumarJhaSDE2025.pdf'
extracted_text = extract_text_from_pdf(pdf_file_path)
print(extracted_text)
# Added a comment to make an edit

Ravi Kumar Jha
+91 9799838629 | iamravijha057@gmail.com |
https://www.linkedin.com/in/ravi-jha-a08266151/ | https://github.com/ravijha9546 |
https://leetcode.com/u/Ravijha9546/
Greater Noida, Uttar Pradesh - 201310, India
OBJECTIVE
A highly motivated and detail-oriented Computer Science graduate with a strong foundation in software
development, data structures, operating systems such as Linux. Looking for an entry-level Software Development
Engineer (SDE) position to leverage my programming skills, problem-solving abilities, and passion for building
scalable and efficient software solutions. Interested in contributing to innovative projects while continually learning
and growing within a dynamic engineering team.
SKILLS
• Programming Languages:C, C++, JavaScript
• Web Technologies:HTML, CSS, Node.js, Express.js, React
• Cloud Technologies:Amazon Web Services, Microsoft Azure
• DevOps & Version Control:Git, GitHub, Docker, CI/CD , Terraform
• Core Subject:Data Structures and Algorithms,

## Commit and push changes

### Subtask:
Make changes to the notebook and push the updates to the linked GitHub repository.


## Summary:

### Data Analysis Key Findings

*   The primary task of pushing a Google Colab notebook to a GitHub repository involves several steps that require manual interaction with the Google Colab interface, such as linking accounts, saving copies to GitHub, and managing revisions.
*   Automated execution within the provided environment could not complete the necessary steps for linking Colab and GitHub accounts.
*   Saving a copy of the notebook directly to GitHub using the "Save a copy in GitHub" feature necessitates manual user intervention.
*   While a code change was successfully made to the notebook, the subsequent steps of saving the revision, viewing revision history, and pushing changes to GitHub could not be executed programmatically.

### Insights or Next Steps

*   Future tasks involving integration between Google Colab and GitHub should acknowledge the manual steps required within the Colab UI.
*   Consider if there are alternative methods or tools outside of the current environment that could facilitate programmatic pushing of changes from Colab to GitHub, potentially using GitHub APIs if applicable.
